<h4>Initial Configuration</h4>

In [1]:
# Open the configuration file
import yaml

with open("credentials.yaml") as f:
    credentials = yaml.safe_load(f)
    
    CLUSTER_ENDPOINT = credentials["CLUSTER_ENDPOINT"]
    TOKEN = credentials["TOKEN"]

In [2]:
from pymilvus import MilvusClient, DataType

# Set up a Milvus client
client = MilvusClient(
    uri = CLUSTER_ENDPOINT,
    token = TOKEN 
)

from pymilvus import DataType

# Create schema
schema = MilvusClient.create_schema(
    auto_id = False,
    enable_dynamic_field = True,
)

# Add fields to schema
schema.add_field(field_name = "id", datatype = DataType.INT64, is_primary = True)
schema.add_field(field_name = "vector", datatype = DataType.FLOAT_VECTOR, dim = 5)
schema.add_field(field_name = "color", datatype = DataType.VARCHAR, max_length = 1000)
schema.add_field(field_name = "color_tag", datatype = DataType.VARCHAR, max_length = 1000)

index_params = client.prepare_index_params()

index_params.add_index(
    field_name = "vector", 
    index_type = "AUTOINDEX",
    metric_type = "IP",
    params = { "nlist": 128 }
)

import time

# Create a collection with the index loaded simultaneously
client.create_collection(
    collection_name = "quick_setup_collection",
    schema = schema,
    index_params = index_params
)

time.sleep(5)

res = client.get_load_state(
    collection_name = "quick_setup_collection"
)

print(res)

{'state': <LoadState: Loaded>}


<h4>Random Data Generation</h4>

In [3]:
import random

# Insert randomly generated vectors 
colors = ["green", "blue", "yellow", "red", "black", "white", "purple", "pink", "orange", "brown", "grey"]
data = []

for i in range(1000):
    current_color = random.choice(colors)
    data.append({
        "id": i,
        "vector": [ random.uniform(-1, 1) for _ in range(5) ],
        "color": current_color,
        "color_tag": f"{current_color}_{str(random.randint(1000, 9999))}"
    })

insert_status = client.insert(
    collection_name = "quick_setup_collection",
    data = data
)

print(insert_status)

{'insert_count': 1000, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

<h4>Generate Partitions</h4>

In [4]:
# Create blue and red partitions
client.create_partition(
    collection_name = "quick_setup_collection",
    partition_name = "red"
)

client.create_partition(
    collection_name = "quick_setup_collection",
    partition_name = "blue"
)

# Insert data into partitions
red_data = [ {"id": i, "vector": [ random.uniform(-1, 1) for _ in range(5) ], "color": "red", "color_tag": f"red_{str(random.randint(1000, 9999))}" } for i in range(500) ]
blue_data = [ {"id": i, "vector": [ random.uniform(-1, 1) for _ in range(5) ], "color": "blue", "color_tag": f"blue_{str(random.randint(1000, 9999))}" } for i in range(500) ]

insert_status = client.insert(
    collection_name = "quick_setup_collection",
    data = red_data,
    partition_name = "red"
)

print(insert_status)

insert_status = client.insert(
    collection_name = "quick_setup_collection",
    data = blue_data,
    partition_name = "blue"
)

print(insert_status)

{'insert_count': 500, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

<h4>Single Vector Search</h4>

In [5]:
import json
# Single vector search
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

search_status = client.search(
    collection_name = "quick_setup_collection",
    data = [query_vector],
    limit = 3, # The number of results to return
    search_params = {"metric_type": "IP", "params": {"level": 1}}
)

search_result = json.dumps(search_status, indent = 4)
print(search_result)

[
    [
        {
            "id": 894,
            "distance": 1.820735216140747,
            "entity": {}
        },
        {
            "id": 318,
            "distance": 1.7348827123641968,
            "entity": {}
        },
        {
            "id": 199,
            "distance": 1.6810312271118164,
            "entity": {}
        }
    ]
]


<h4>Batch Vector Search</h4>

In [6]:
# Batch-vector search
bulk_query_vectors = [
    [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592],
    [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104]
] # A list of two vectors

bulk_search_status = client.search(
    collection_name = "quick_setup_collection",
    data = bulk_query_vectors, 
    limit = 2,
    search_params = {"metric_type": "IP", "params": {"level": 1}}
)


bulk_result = json.dumps(bulk_search_status, indent = 4)
print(bulk_result)

[
    [
        {
            "id": 894,
            "distance": 1.820735216140747,
            "entity": {}
        },
        {
            "id": 318,
            "distance": 1.7348827123641968,
            "entity": {}
        }
    ],
    [
        {
            "id": 287,
            "distance": 1.8029110431671143,
            "entity": {}
        },
        {
            "id": 253,
            "distance": 1.6727240085601807,
            "entity": {}
        }
    ]
]


<h4>Partition Search</h4>

In [7]:
# Search within a partition
partition_query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

partition_search_status = client.search(
    collection_name = "quick_setup_collection",
    data = [partition_query_vector],
    limit = 5,
    search_params = {"metric_type": "IP", "params": {"level": 1}}, # level -> search precision level (higher > more accurate > more computation time)
    partition_names = ["red"]
)


partition_result = json.dumps(partition_search_status, indent = 4)
print(partition_result)

[
    [
        {
            "id": 318,
            "distance": 1.7348827123641968,
            "entity": {}
        },
        {
            "id": 199,
            "distance": 1.6810312271118164,
            "entity": {}
        },
        {
            "id": 443,
            "distance": 1.646945595741272,
            "entity": {}
        },
        {
            "id": 300,
            "distance": 1.4745960235595703,
            "entity": {}
        },
        {
            "id": 47,
            "distance": 1.4011565446853638,
            "entity": {}
        }
    ]
]


<h4>Configure Output Fields</h4>

In [8]:
# Search with output fields
output_query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

output_search_status = client.search(
    collection_name = "quick_setup_collection",
    data = [output_query_vector],
    limit = 5,
    search_params = {"metric_type": "IP", "params": {"level": 1}},
    output_fields = ["color"]
)

output_result = json.dumps(output_search_status, indent = 4)
print(output_result)

[
    [
        {
            "id": 894,
            "distance": 1.820735216140747,
            "entity": {
                "color": "yellow"
            }
        },
        {
            "id": 318,
            "distance": 1.7348827123641968,
            "entity": {
                "color": "red"
            }
        },
        {
            "id": 199,
            "distance": 1.6810312271118164,
            "entity": {
                "color": "red"
            }
        },
        {
            "id": 274,
            "distance": 1.663917899131775,
            "entity": {
                "color": "blue"
            }
        },
        {
            "id": 694,
            "distance": 1.6502282619476318,
            "entity": {
                "color": "grey"
            }
        }
    ]
]


<h4>Like Operator</h4>

In [9]:
# Filter with "like" operator and prefix wildcard
like_query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

like_search_status = client.search(
    collection_name = "quick_setup_collection",
    data = [like_query_vector],
    limit = 5,
    search_params = {"metric_type": "IP", "params": {"level": 1}},
    filter = 'color_tag like "red%"',
    output_fields = ["color_tag"]
)

like_result = json.dumps(like_search_status, indent = 4)
print(like_result)

[
    [
        {
            "id": 318,
            "distance": 1.7348827123641968,
            "entity": {
                "color_tag": "red_3227"
            }
        },
        {
            "id": 199,
            "distance": 1.6810312271118164,
            "entity": {
                "color_tag": "red_5896"
            }
        },
        {
            "id": 443,
            "distance": 1.646945595741272,
            "entity": {
                "color_tag": "red_7454"
            }
        },
        {
            "id": 300,
            "distance": 1.4745960235595703,
            "entity": {
                "color_tag": "red_3931"
            }
        },
        {
            "id": 47,
            "distance": 1.4011565446853638,
            "entity": {
                "color_tag": "red_2913"
            }
        }
    ]
]


In [11]:
import json

# Infix match on color field
infix_query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

infix_search_status = client.search(
    collection_name = "quick_setup_collection", # Replace with the actual name of your collection
    data = [infix_query_vector],
    limit = 5, # Max. number of search results to return
    search_params = {"metric_type": "IP", "params": {"level": 1}}, # Search parameters
    output_fields = ["color_tag"], # Output fields to return
    filter = 'color like "%l%"' # Filter on color field, infix match on "ll"
)

infix_result = json.dumps(infix_search_status, indent = 4)
print(infix_result)

[
    [
        {
            "id": 894,
            "distance": 1.820735216140747,
            "entity": {
                "color_tag": "yellow_5241"
            }
        },
        {
            "id": 274,
            "distance": 1.663917899131775,
            "entity": {
                "color_tag": "blue_8096"
            }
        },
        {
            "id": 390,
            "distance": 1.6487438678741455,
            "entity": {
                "color_tag": "blue_7443"
            }
        },
        {
            "id": 12,
            "distance": 1.6417144536972046,
            "entity": {
                "color_tag": "blue_6963"
            }
        },
        {
            "id": 349,
            "distance": 1.5990583896636963,
            "entity": {
                "color_tag": "blue_2153"
            }
        }
    ]
]


<h4>Range Search</h4>

In [12]:
# Range search
range_search_params = {
    "metric_type": "IP",
    "params": {
        "radius": 0.8, # Radius of the search circle
        "range_filter": 1.0 # Range filter to filter out vectors that are not within the search circle
    }
}

range_query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

range_search_status = client.search(
    collection_name = "quick_setup_collection",
    data = [range_query_vector],
    limit = 3,
    search_params = range_search_params, # Search parameters
    output_fields = ["color_tag"], # Output fields to return
)

range_result = json.dumps(range_search_status, indent=4)
print(range_result)

[
    [
        {
            "id": 20,
            "distance": 0.9918922781944275,
            "entity": {
                "color_tag": "red_6826"
            }
        },
        {
            "id": 348,
            "distance": 0.9914668798446655,
            "entity": {
                "color_tag": "blue_4829"
            }
        },
        {
            "id": 458,
            "distance": 0.9879510402679443,
            "entity": {
                "color_tag": "yellow_2317"
            }
        }
    ]
]


<h4>Group Search</h4>

In [16]:
# Load data into collection
client.load_collection("quick_setup_collection") # Collection name

# Group search results
res = client.search(
    collection_name = "quick_setup_collection", # Collection name
    data = [[0.14529211512077012, 0.9147257273453546, 0.7965055218724449, 0.7009258593102812, 0.5605206522382088]], # Query vector
    search_params = {
        "metric_type": "IP",
        "params": {"level": 1},
    },
    limit = 10,
    group_by_field = "id",
    output_fields = ["id", "color", "color_tag"]
)

# Retrieve the values in the `doc_id` column
doc_ids = [result['entity']['color'] for result in res[0]]

print(doc_ids)

['blue', 'purple', 'black', 'blue', 'blue', 'red', 'purple', 'red', 'blue', 'red']


In [17]:
# Drop collection
client.drop_collection(
    collection_name = "quick_setup_collection",
)